In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

from torchvision.models import resnet50, ResNet50_Weights
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import resnet
from custom_datasets import CIFAR10C
import wandb
import numpy as np
import random

In [9]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)


In [22]:


# val_loader = torch.utils.data.DataLoader(
#     datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
#         transforms.ToTensor(),
#         normalize,
#     ])),
#     batch_size=128, shuffle=False,
#     num_workers=args.workers, pin_memory=True)

#load imagenet test set

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(root='data/Tiny-ImageNet-C/shot_noise/1/', transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=4, pin_memory=True)

In [23]:
from trainer import AverageMeter

model.eval()
model.cuda()


entropies = AverageMeter()
with torch.no_grad():
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()
        input_var = input.cuda()
        target_var = target.cuda()

        # compute output
        output = model(input_var)
        output = torch.nn.functional.softmax(output, dim=-1)
        entropy = -torch.sum(output * torch.log(output), dim=-1)
        entropies.update(entropy.mean().item(), input.size(0))

print(entropies.avg)

5.124798268890381


In [24]:
# 1 - 5.124798268890381
# 2 - 5.294856476593018
# 3 - 5.41798617401123
# 4 - 5.474362422180175
# 5 - 5.566304541778565
output.shape

torch.Size([16, 1000])

In [25]:
np.log(1/1000)

-6.907755278982137

In [26]:
np.log(1)

0.0